In [121]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from scraper_api import ScraperAPIClient
client = ScraperAPIClient('4a7d740ef11cdfa724f5d93af70c9a65')

def get_max_page(url,client):
    r = client.get(url)
    soup = BeautifulSoup(r.text)
    last_page_page = soup.findAll('ul',{'class':'SearchPagination-links clearfix'})[0].findAll('li')[-2].text
    return int(last_page_page)

def scrape_99co(max_page,client):
    main = pd.DataFrame()
    for i in range(1,max_page+1):
        try:
            URL = 'https://www.99.co/singapore/sale/hdb?page_num='+str(i)
            r = client.get(URL)
            soup = BeautifulSoup(r.text)
            t_s = soup.findAll('div',{'style':'max-height:270px;opacity:1'})
        except:
            continue

        for t in t_s:
            try:
                detail_1 = t.find('div',{'class':'_1w7Su'})
                address = detail_1.find('h3',{'class':'_2ncUp _1T6Ql'}).text
                postal = detail_1.findAll('p',{'class':'x0JSc TMSN-'})[0].text.split('·')[0].strip()
                # years = detail_1.findAll('p',{'class':'x0JSc TMSN-'})[-1].text.split('·')[-1].split(' ')[1]
                year_built = detail_1.findAll('p',{'class':'x0JSc TMSN-'})[-1].text.split('·')[0].split(' ')[0]

                detail_2 = t.find('div',{'class':'QyDYr'})
                price = detail_2.find('h3',{'class':'_2ncUp _1T6Ql'}).text
                bedrooms = detail_2.findAll('p',{'class':'x0JSc TMSN-'})[0].text.split()[0]
                bathrooms = detail_2.findAll('p',{'class':'x0JSc TMSN-'})[0].text.split()[1]
                floor_area = detail_2.findAll('p',{'class':'x0JSc TMSN-'})[-1].text.split('/')[-1].strip().split()[0]

                link = 'https://www.99.co' + t.find('a',{'class':'_17qrb _3uV4l'})['href']
                temp = pd.DataFrame([[address,link,price,bedrooms,bathrooms,floor_area,year_built]])
                main = main.append(temp)
            except:
                continue
    main.columns = ['address','link','price','bedrooms','bathrooms','floor_area','year_built']
    main['date_scraped'] = datetime.now()
    return main

In [110]:
url = 'https://www.99.co/singapore/sale/hdb?page_num=1'
max_page = get_max_page(url,client)

In [120]:
max_page

228

In [122]:
df = scrape_99co(max_page,client)

In [123]:
df.to_csv('99co.csv',index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7672 entries, 0 to 0
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   address       7672 non-null   object        
 1   link          7672 non-null   object        
 2   price         7672 non-null   object        
 3   bedrooms      7672 non-null   object        
 4   bathrooms     7672 non-null   object        
 5   floor_area    7672 non-null   object        
 6   year_built    7672 non-null   object        
 7   date_scraped  7672 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(7)
memory usage: 539.4+ KB
